# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [3]:
obj = {
    "website": "https://github.com",
    "password": "qwerty213",
    "login": "vasya_pupkin"
}

for key, value in obj.items():
    print("{} = {}".format(key, value))

website = https://github.com
password = qwerty213
login = vasya_pupkin


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
import pandas as pd
import re

obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])

pattern = re.compile(r'(?:ПМ)?\s?\d+-\d', re.I)
for item in obj:
    print(f'{item} : {pattern.findall(item)}')

Евгения гр.ПМ19-1 : ['ПМ19-1']
Илья пм 20-4 : ['пм 20-4']
Анна 20-3 : [' 20-3']


3. Разбейте текст формулировки задачи 2 на слова.

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VareNiC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:
text = "Написать регулярное выражение,которое позволит найти номера групп студентов."
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokenizer.tokenize(text)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [9]:
import pandas as pd

In [10]:
recipes = pd.read_csv('recipes_sample.csv', on_bad_lines="skip", engine='python')

In [11]:
random_rows = recipes[['id', 'minutes']].sample(n=5)

id_width = random_rows['id'].apply(str).apply(len).max()
time_width = random_rows['minutes'].apply(str).apply(len).max()
if time_width < len('minutes'):
  time_width = len('minutes')

print("| {:^{id_width}} | {:^{time_width}} |".format("id", "minutes", id_width=id_width, time_width=time_width))
print("|" + "-"*(id_width+2) + "|" + "-"*(time_width+2) + "|")

for _, row in random_rows.iterrows():
    print("| {:^{id_width}} | {:^{time_width}} |".format(row['id'], row['minutes'], id_width=id_width, time_width=time_width))


|   id   | minutes |
|--------|---------|
| 71531  |   65    |
| 169484 |   40    |
| 189307 |   16    |
| 186900 |   10    |
| 296821 |   55    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1.
2. Шаг 2.
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [12]:
import xml.etree.ElementTree as ET

steps = ET.parse('steps_sample.xml')
print(steps)


In [23]:
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

recipes_dct = {}

def show_info(recipe_id):
    global recipes_dct 

    recipes = pd.read_csv('recipes_sample.csv', on_bad_lines="skip", engine='python')
    with open ("steps_sample.xml", 'r', encoding = 'utf-8') as fp:
      data = BeautifulSoup(fp,'xml')
    
    for recipe in data.find_all('recipe'):
      r_id = int(recipe.id.contents[0])
      steps = [step.text for step in recipe.find('steps').find_all('step')]
      recipes_dct[r_id] = steps

    steps_text = ""
    for step_num, step_text in enumerate(recipes_dct[recipe_id]):
      steps_text += f'{step_num + 1}. {step_text.capitalize()}\n'

    recipe = recipes.loc[recipes['id'] == recipe_id]
    if recipe.empty:
        return f"No recipe found with id {recipe_id}"
    
    title = recipe['name'].values[0].title()
    author = recipe['contributor_id'].values[0]
    time = recipe['minutes'].values[0]
    
    output = f'"{title}"\n\n{steps_text}----------\nАвтор: {author}\nСреднее время приготовления: {time} минут'
    return output


In [24]:
print(show_info(170895))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [25]:
import re

In [26]:
obj = pd.Series(recipes_dct[25082])

patt = re.compile(r'\d{1,}\s(?:hours|hour|minutes|minute)', re.I)
for item in obj:
    if patt.findall(item) == []:
        continue
    print(patt.findall(item))

['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [32]:
k = 0
kostil = -1
recipes_nmb = []
patt = re.compile(r'^this[\w\s]*,\s?but\b', re.I)
for item in recipe:
    kostil += 1
    if type(item) == str:
        if patt.findall(item) == []:
            continue
        #print(patt.findall(item))
        k += 1
        recipes_nmb.append(kostil)
    
print("Количество отзывов с таким шаблоном: ", k)

Количество отзывов с таким шаблоном:  132


In [33]:
#выводим на три описания, текст виден полностью
for i in range(3):
    print(f'''{i+1}. {recipes['description'].iloc[recipes_nmb[i]]}\n''')

1. this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.

2. this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

3. this is kind of similar to some of the other versions out there, but it is the best and easiest i have found



5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [49]:
steps = recipes_dct[72367]
patt = re.compile(r'\s+/\s+')

for i in range(len(steps)):
    steps[i] = patt.sub('/', steps[i])
    
for i in range(len(steps)):
    print(f'{i+1}. {steps[i]}')

1. mix butter , flour , 1/3 c
2. sugar and 1-1/4 t
3. vanilla
4. press into greased 9" springform pan
5. mix cream cheese , 1/4 c
6. sugar , eggs and 1/2 t
7. vanilla beating until fluffy
8. pour over dough
9. combine apples , 1/3 c
10. sugar and cinnamon
11. arrange on top of cream cheese mixture and sprinkle with almonds
12. bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [85]:
keys = list(recipes_dct.keys())
unique = set()
for key, step_l in recipes_dct.items():
    for text in step_l:
        if text:
            tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
            tok_l = tokenizer.tokenize(text)
            unique.update(tok.lower() for tok in tok_l)

line = []
for i in range(len(keys)):
    for j in range(len(recipes_dct[keys[i]])):
        
        if type(recipes_dct[keys[i]][j]) != str:
            continue
        text = recipes_dct[keys[i]][j]
        nltk.word_tokenize(text)
        tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
        t = tokenizer.tokenize(text)
        
        for l in range(len(t)):
            if t[l].lower() in stroka:
                continue
            line.append(t[l])
            
f = list(set(line))
f_bool = [f[i].isalpha() for i in range(len(f))]
alph_tok_l = [tok for tok in unique if tok.isalpha()]
len(alph_tok_l)

15114

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [87]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VareNiC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [95]:
def split_sentences(description):
    return nltk.sent_tokenize(str(description))


recipes['sentences'] = recipes['description'].apply(split_sentences)
recipes_sorted = recipes.sort_values(by=['sentences'], key=lambda x: x.str.len(), ascending=False)
top_5_longest = recipes_sorted.head(5)
for index, row in top_5_longest.iterrows():
    print(index, '\n', row['description'])
    print('\n')


18408 
 this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.  it makes a delicious filling also between the layers of cakes and under fondant icing.  you can make roses but it takes 3 or more days to dry them depending on the humidity. 

there are many versions of “buttercream” icing. some are made with eggs and all butter.  some varieties, you have to cook your sugar to a softball stage.  others are 100% shortening or a combination of shortening and butter.

each decorator has his or her favorite.  i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch. but…. i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.  it melts!  a greasy puddle of melted icing on a cake plate is not something i want to look at or eat.  

your top notch decorators have a few options we d

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [103]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\VareNiC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [119]:
def get_parts_of_speech(sentence):
    words = word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)
    return pos_tags

recipe_name = recipes.loc[recipes['id'] == 241106]['name'].values[0]
pos_tags = get_parts_of_speech(recipe_name)

tags, line = '', ''
for tag in pos_tags:
    tags += ' ' * (len(tag[0]) // 2 - 1) + tag[1] + ' ' * (len(tag[0]) // 2)
    line += tag[0] + ' '

print(tags)
print(line)


   JJ      NNS    IN     NNS     VBP    JJ   CC  JJ    NNS   
eggplant steaks with chickpeas feta cheese and black olives 
